**Version**
** Revision Number **
0  This is just an increasing number. If you make a change increase the number.

**Group members**
Please add your full name and student id(?)
Qinyuan,
Eamon Maloney,
Ibrahim Saana Aminu 25381993,
Des Powell 9513833,
Terence Coffey 15223124

Develop a program in Python using the Keras Neural Network to implement a classifier fot the NMIST handwritten digits database. It is recommended to use a convolution neural network.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

**Information sources to assist with completing this project goes here.**

[How to create a pdf file for Colab notebook](https://saturncloud.io/blog/convert-google-colab-notebook-to-pdf-html/#:~:text=accessible%20and%20shareable.-,Converting%20Colab%20Notebooks%20to%20PDF,to%20various%20formats%2C%20including%20PDF\)

[Getting started with Keras](https://keras.io/getting_started/)
[Keras for Engineers](https://keras.io/getting_started/intro_to_keras_for_engineers/)

[Alternative solutions](https://en.m.wikipedia.org/wiki/MNIST_database)
[Explanation of some of the layers](https://divingintogeneticsandgenomics.com/post/deep-learning-with-keras-using-mnst-dataset/)





In [ ]:
## Install to allow you to convert this Notebook to a pdf file
! pip install nbconvert
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc

Create the pdf

In [ ]:
!jupyter nbconvert --to pdf Assign1_MNIST_Keras.ipynb

Information about the project
**MNIST character recognition**
Description goes here.

In [ ]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.regularizers import l2
from keras.callbacks import EarlyStopping

import numpy as np
from sklearn.metrics import confusion_matrix

# just one for now....
NUM_RUNS = 1
dropout_rate_list = [0.2, 0.5]
batch_size_list = [32, 64]
val_split_list = [0.2]
learn_rate_list = [0.0001, 0.01]
kernel_size_list = [3, 5]
filter_size_list = [15]
seed_value_list = [42,50]
regularizer_list = [0.0, 0.000001]

# tf.debugging.set_log_device_placement(True)

num_classes = 10
input_shape = (28, 28, 1)
epoch_proj1 = 30

early_stopping = EarlyStopping(monitor='val_loss', patience=6)
fit_callbacks = [early_stopping]

# Define the model
for num_runs in range(NUM_RUNS):
    for dropout_rate in dropout_rate_list:
        for b_size in batch_size_list:
            for val_split in val_split_list:
                for learn_rate in learn_rate_list:
                    for ksize in kernel_size_list:
                        for fsize in filter_size_list:
                            for regularizer in regularizer_list:
                                for seed_value in seed_value_list:
                                
                                    tf.random.set_seed(seed_value)

                                    # Load and pre-process MNIST dataset
                                    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
                                    x_train = x_train.astype("float32") / 255.0
                                    x_test = x_test.astype("float32") / 255.0
                                    y_train = keras.utils.to_categorical(y_train, num_classes=10)
                                    y_test = keras.utils.to_categorical(y_test, num_classes=10)

                                    # MODEL 1
                                    model1 = keras.Sequential(
                                        [
                                            keras.Input(shape=input_shape),
                                            
                                            layers.Conv2D(fsize, kernel_size=(ksize, ksize), activation="relu"),
                                            layers.MaxPooling2D(pool_size=(2, 2)),
                                            
                                            layers.Conv2D(fsize*2, kernel_size=(ksize, ksize), activation="relu"),
                                            layers.MaxPooling2D(pool_size=(2, 2)),
                                            
                                            layers.Flatten(),
                                            layers.Dropout(dropout_rate),
                                            layers.Dense(10, activation="softmax"),
                                        ]
                                    )  
                                    # Compile the model
                                    model1.compile(
                                        loss="categorical_crossentropy",
                                        optimizer=Adam(learning_rate=learn_rate),
                                        metrics=['accuracy', Precision(), Recall()]
                                    )
                                    model1.fit(x_train, y_train, batch_size=b_size, epochs=epoch_proj1, validation_split=val_split) #, callbacks=fit_callbacks)                              
                                    # predict using the test data
                                    predictions1 = model1.predict(x_test)
                                    predicted_labels1 = np.argmax(predictions1, axis=1)
                                    true_labels1 = np.argmax(y_test, axis=1)
                                    conf_matrix1 = confusion_matrix(true_labels1, predicted_labels1)                            
                                    results_model1 = model1.evaluate(x_test, y_test, verbose=0)
                                    num_epochs_run = early_stopping.stopped_epoch if early_stopping.stopped_epoch > 0 else epoch_proj1
                                    print(f'Model1 Results - #Epochs:{num_epochs_run}, Dopout Rate:{dropout_rate},Batch Size:{b_size},Validation Split:{val_split},Learning Rate:{learn_rate},Kernel Size:{ksize},Filter Size:{fsize},Regularizer:{regularizer},Seed: {seed_value} **** Loss:{results_model1[0]}, Accuracy:{results_model1[1]}, Precision:{results_model1[2]}, Recall:{results_model1[3]}')
                                    print(f'Model1 Summary {model1.summary()}')      
                                    print(f'Model1 Confusion Matrix\n{conf_matrix1}')
                                                    

                                    # MODEL 2
                                    model2 = keras.Sequential(
                                        [
                                            keras.Input(shape=input_shape),
                                            
                                            layers.Conv2D(fsize, kernel_size=(ksize, ksize), activation="relu"),
                                            layers.MaxPooling2D(pool_size=(2, 2)),
                                            layers.Dropout(dropout_rate),  # First dropout layer
                                            
                                            layers.Conv2D(fsize*2, kernel_size=(ksize, ksize), activation="relu"),
                                            layers.MaxPooling2D(pool_size=(2, 2)),
                                            layers.Dropout(dropout_rate),  # Second dropout layer
                                            
                                            layers.Flatten(),
                                            layers.Dense(fsize*4, activation="relu"),  # First dense layer
                                            layers.Dense(10, activation="softmax"),  # Second dense (output) layer
                                        ]
                                    )
                                    model2.compile(
                                        loss="categorical_crossentropy",
                                        optimizer=Adam(learning_rate=learn_rate),
                                        metrics=['accuracy', Precision(), Recall()]
                                    )  
                                    model2.fit(x_train, y_train, batch_size=b_size, epochs=epoch_proj1, validation_split=val_split) #, callbacks=fit_callbacks)                              
                                    # predict using the test data - model 3
                                    predictions2 = model2.predict(x_test)
                                    predicted_labels2 = np.argmax(predictions2, axis=1)
                                    true_labels2 = np.argmax(y_test, axis=1)
                                    conf_matrix2 = confusion_matrix(true_labels2, predicted_labels2)                            
                                    results_model2 = model2.evaluate(x_test, y_test, verbose=0)
                                    num_epochs_run = early_stopping.stopped_epoch if early_stopping.stopped_epoch > 0 else epoch_proj1
                                    print(f'Model2 Results - #Epochs:{num_epochs_run}, Dopout Rate:{dropout_rate},Batch Size:{b_size},Validation Split:{val_split},Learning Rate:{learn_rate},Kernel Size:{ksize},Filter Size:{fsize},Regularizer:{regularizer},Seed: {seed_value} **** Loss:{results_model2[0]}, Accuracy:{results_model2[1]}, Precision:{results_model2[2]}, Recall:{results_model2[3]}')
                                    print(f'Model2 Summary {model2.summary()}')      
                                    print(f'Model2 Confusion Matrix\n{conf_matrix2}')                   

                                    
                                    # MODEL 3 - lecture notes recommendation
                                    model3 = keras.Sequential(
                                        [
                                            keras.Input(shape=input_shape),
                                            
                                            layers.Conv2D(15, kernel_size=(ksize, ksize), padding='same', activation="relu", kernel_initializer=HeNormal(seed=seed_value)),
                                            layers.MaxPooling2D(pool_size=(2, 2)),
                                            
                                            
                                            layers.Conv2D(30, kernel_size=(ksize, ksize), padding='same', activation="relu", kernel_initializer=HeNormal(seed=seed_value), kernel_regularizer=l2(regularizer)),
                                            layers.MaxPooling2D(pool_size=(2, 2)),
                                                                                
                                            layers.Flatten(),
                                            
                                            layers.Dropout(dropout_rate),
                                            layers.Dense(256, activation="relu", kernel_regularizer=l2(regularizer)),   
                                            
                                            layers.Dropout(dropout_rate),
                                            layers.Dense(10, activation="softmax"),  # Second dense (output) layer
                                        ]
                                    )
                                    model3.compile(
                                        loss="categorical_crossentropy",
                                        optimizer=Adam(learning_rate=learn_rate),
                                        metrics=['accuracy', Precision(), Recall()]
                                    )                                                                                                                              
                                    model3.fit(x_train, y_train, batch_size=b_size, epochs=epoch_proj1, validation_split=val_split) #, callbacks=fit_callbacks)                              
                                    # predict using the test data - model 3
                                    predictions3 = model2.predict(x_test)
                                    predicted_labels3 = np.argmax(predictions3, axis=1)
                                    true_labels3 = np.argmax(y_test, axis=1)
                                    conf_matrix3 = confusion_matrix(true_labels3, predicted_labels3)                            
                                    results_model3 = model3.evaluate(x_test, y_test, verbose=0)
                                    num_epochs_run = early_stopping.stopped_epoch if early_stopping.stopped_epoch > 0 else epoch_proj1
                                    print(f'Model3 Results - #Epochs:{num_epochs_run}, Dopout Rate:{dropout_rate},Batch Size:{b_size},Validation Split:{val_split},Learning Rate:{learn_rate},Kernel Size:{ksize},Filter Size:{fsize},Regularizer:{regularizer},Seed: {seed_value} **** Loss:{results_model3[0]}, Accuracy:{results_model3[1]}, Precision:{results_model3[2]}, Recall:{results_model3[3]}')
                                    print(f'Model3 Summary {model3.summary()}')      
                                    print(f'Model3 Confusion Matrix\n{conf_matrix3}')
                                        
                            

2024-10-28 22:50:45.958690: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-28 22:50:45.974845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730155845.993992  345205 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730155845.999398  345205 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-28 22:50:46.018534: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Epoch 1/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.5648 - loss: 1.4423 - precision: 0.8023 - recall: 0.2715 - val_accuracy: 0.9228 - val_loss: 0.2917 - val_precision: 0.9518 - val_recall: 0.8861
Epoch 2/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.9008 - loss: 0.3320 - precision: 0.9327 - recall: 0.8697 - val_accuracy: 0.9457 - val_loss: 0.1915 - val_precision: 0.9620 - val_recall: 0.9333
Epoch 3/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 15s 10ms/step - accuracy: 0.9300 - loss: 0.2296 - precision: 0.9464 - recall: 0.9140 - val_accuracy: 0.9568 - val_loss: 0.1512 - val_precision: 0.9687 - val_recall: 0.9478
Epoch 4/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.9437 - loss: 0.1823 - precision: 0.9562 - recall: 0.9322 - val_accuracy: 0.9631 - val_loss: 0.1280 - val_precision: 0.9733 - val_recall: 0.9563
Epoch 5/30
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 16s 11ms/step - accuracy: 0.9539 - loss: 0.1558 - precision: 0.9638 - recall: 0.9446 - val_accuracy: 0.96

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 15)     │           150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 15)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 30)     │         4,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 30)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 750)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 750)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         7,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 35,222 (137.59 KB)

...

 Total params: 149,702 (584.78 KB)

 Trainable params: 49,900 (194.92 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 99,802 (389.86 KB)

Model2 Summary None
Model2 Confusion Matrix
[[ 972    1    0    0    1    0    3    0    2    1]
 [   0 1130    1    1    0    0    2    1    0    0]
 [   0    0 1018    0    1    0    1    4    8    0]
 [   0    0    1 1004    0    3    0    2    0    0]
 [   0    1    0    0  973    0    2    2    0    4]
 [   1    0    0   15    0  874    0    0    1    1]
 [   3    3    0    1    2    4  944    0    1    0]
 [   0    3    5    1    0    0    0 1008    6    5]
 [   0    0    0    8    2    3    1    0  958    2]
 [   0    2    0    5   11    1    0   11    1  978]]
Epoch 1/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.8799 - loss: 0.4343 - precision_83: 0.9217 - recall_83: 0.8411 - val_accuracy: 0.9736 - val_loss: 0.0919 - val_precision_83: 0.9772 - val_recall_83: 0.9693
Epoch 2/30
750/750 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.9682 - loss: 0.1129 - precision_83: 0.9725 - recall_83: 0.9645 - val_accuracy: 0.9778 - val_loss: 0.0799 - val_precision_83: 0.9814 - v

Application using Keras for MNIST Digit Classifier using 2 convolution layers , 2 drop out layers and 2 accumulation layers. Split the dataset into training and test data in the ratio 70 percent and 30 percent respectively.

Display accuracy of Testing and Validation dataset

Discuss the Project especially
- How you choose to tackle it
- What design decisions you made
- What the results are like
- What you might do better/differently next time you had to tackle a similar project
- If plots are called for they should be in your code and in your report.
- Marks for neat well designed code with appropriate level of comments
- neat logically laid out and informative reports.
- Provide classification accuracy for the training and test data. The test data should be split in the ration 70 to 80 and the baance for validation.


